In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import gluonnlp as nlp
from tqdm import tqdm

from transformers import AutoModel, AutoTokenizer, RobertaTokenizer, AutoModelForSequenceClassification
from transformers import AutoModelForMaskedLM, RobertaForSequenceClassification

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

2023-02-06 02:57:14.608557: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [2]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import re

In [3]:
import os

In [4]:
import gc
import torch

gc.collect()
torch.cuda.empty_cache()

In [5]:
gpu_id = 0

In [6]:
device = (
        torch.device("cpu")
        if gpu_id < 0
        else torch.device("cuda:%d" % gpu_id)
    )


In [7]:
# HUGGINGFACE_MODEL_PATH = "rurupang/roberta-base-finetuned-sts"
# HUGGINGFACE_MODEL_PATH = "klue/roberta-base"
HUGGINGFACE_MODEL_PATH = "klue/roberta-small"

In [8]:
# roberta_tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

tokenizer = AutoTokenizer.from_pretrained(HUGGINGFACE_MODEL_PATH)

# model = AutoModelForSequenceClassification.from_pretrained("rurupang/roberta-base-finetuned-sts")
# tokenizer = AutoTokenizer.from_pretrained("rurupang/roberta-base-finetuned-sts", use_fast=True)

In [9]:
train_df = pd.read_csv("/home/keti/Desktop/eunbyeol/Workspace/Data-planet/ETRI/train.csv")
test_df = pd.read_csv("/home/keti/Desktop/eunbyeol/Workspace/Data-planet/ETRI/test.csv")

train_df = train_df.dropna(axis=0)
test_df = test_df.dropna(axis=0)

In [10]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
label_encoder.fit(train_df['label'])
num_labels = len(label_encoder.classes_)

train_df['encoded_label'] = np.asarray(label_encoder.transform(train_df['label']), dtype=np.int32)
train_df.head()


,title,description,url,data_type,source,ori_label,ori_source,data_platform_id,label,encoded_label
0,서울시 강동구 집단급식소식품판매업 인허가 정보,강동구의 집단급식소를 대상으로 식품을 판매하는 업소정보* 좌표안내 : 중부원점TM(...,https://data.seoul.go.kr/dataList/OA-18051/S/1...,SHEETOpenAPI,강동구 스마트도시정책관 빅데이터담당관,보건,서울데이터광장,3,보건의료,5
1,경상남도 함양군_지방세 납부 현황,신용카드 가상계좌 등 지방세 납부매체별 납부현황으로 납부년도 납부매체 납부매체전자고...,https://www.data.go.kr/data/15079319/fileData.do,CSV,공공데이터포털,경제금융,빅데이터지도,1,경제금융,0
2,부산광역시 남구_하천점용현황_20210927,부산광역시 남구 하천 점용 현황에 대한 데이터로 유형 하천명 소재지 면적 점용시작...,https://www.data.go.kr/data/15047963/fileData.do,텍스트,부산광역시 남구,일반공공행정,공공데이터포털,0,행정법률,12
4,한국주택금융공사_일정관리관련내역,한국주택금융공사 채권관리부 일정관리관련 내역이 포함되어 있습니다. (해당 부서의 업...,https://www.data.go.kr/data/15072798/fileData.do,CSV,공공데이터포털,환경자원,빅데이터지도,1,환경기상,13
5,충청남도_노인의료복지시설 현황_20211231,2019년 노인 요양시설 및 요양공동생활가정에 관한 지역 시설 종류 시설명 연락처 ...,https://www.data.go.kr/data/15044627/fileData.do,텍스트,충청남도,사회복지,공공데이터포털,0,사회복지,6


In [11]:
test_df['encoded_label'] = np.asarray(label_encoder.transform(test_df['label']), dtype=np.int32)
test_df.head()

,title,description,url,data_type,source,ori_label,ori_source,data_platform_id,label,encoded_label
0,서울특별시 동작구_체육시설업 현황_20220719,이 데이터는 서울특별시 동작구 체육시설업 현황에 관한 것입니다. 이 데이터에는 업종...,https://www.data.go.kr/data/15016523/fileData.do,텍스트,서울특별시 동작구,문화체육관광,공공데이터포털,0,문화관광,4
1,국가보훈처_세종특별자치시 읍면동별 보훈대상자 인원현황,세종특별자치시 시군구별 보훈대상자 인원현황 자료1. 적용 대상 국가유공자는 「국가유...,https://www.data.go.kr/data/15098581/fileData.do,CSV,공공데이터포털,행정법률,빅데이터지도,1,행정법률,12
2,한국주택금융공사_계좌별누적회수이자정보(회계기간_해당결산월말)_20200901,한국주택금융공사의 계좌별누적회수이자정보에 대한 데이터로 유동화계획코드 인수코드 회수...,https://www.data.go.kr/data/15073239/fileData.do,텍스트,한국주택금융공사,사회복지,공공데이터포털,0,사회복지,6
3,법무부_56(국적) 연도별 국적 상실 이탈 처리 현황,대한민국 국적을 상실하거나 이탈 처리된 자의 현황을 연도별로 제공(연도 유형(상실 ...,https://www.data.go.kr/data/15100048/fileData.do,CSV,공공데이터포털,행정법률,빅데이터지도,1,행정법률,12
4,인천광역시 서구_쓰레기 무단투기 감시용 카메라(CCTV) 설치 현황_20220824,인천광역시 서구의 쓰레기 무단투기 감시용 카메라(CCTV) 설치 현황에 관한 데이터...,https://www.data.go.kr/data/15105069/fileData.do,텍스트,인천광역시 서구,환경,공공데이터포털,0,환경기상,13


In [12]:
train_texts = train_df["description"].to_list() # Features (not-tokenized yet)
train_labels = torch.nn.functional.one_hot(torch.tensor(train_df["encoded_label"].to_list())) # Labels

test_texts = test_df["description"].to_list() # Features (not-tokenized yet)
test_labels = torch.nn.functional.one_hot(torch.tensor(test_df["encoded_label"].to_list())) # Labels


In [13]:
test_labels.size()

torch.Size([11501, 14])

In [14]:
train_encodings = tokenizer(train_texts, truncation=True, padding=True, return_tensors="pt")
test_encodings = tokenizer(test_texts, truncation=True, padding=True, return_tensors="pt")

In [15]:
train_encodings

{'input_ids': tensor([[    0, 19647,  2079,  ...,     1,     1,     1],
        [    0, 11776,  7024,  ...,     1,     1,     1],
        [    0,  3902, 16955,  ...,     1,     1,     1],
        ...,
        [    0,  4364, 20000,  ...,     1,     1,     1],
        [    0, 29890,  4856,  ...,     1,     1,     1],
        [    0,  9975,  2155,  ...,     1,     1,     1]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}

In [16]:
train_encodings.input_ids

tensor([[    0, 19647,  2079,  ...,     1,     1,     1],
        [    0, 11776,  7024,  ...,     1,     1,     1],
        [    0,  3902, 16955,  ...,     1,     1,     1],
        ...,
        [    0,  4364, 20000,  ...,     1,     1,     1],
        [    0, 29890,  4856,  ...,     1,     1,     1],
        [    0,  9975,  2155,  ...,     1,     1,     1]])

In [17]:
from torch.utils.data import DataLoader, TensorDataset

train_data = TensorDataset(train_encodings.input_ids, train_encodings.attention_mask, train_labels)
train_dataloader = DataLoader(train_data, batch_size=8)

test_data = TensorDataset(test_encodings.input_ids, test_encodings.attention_mask, test_labels)
test_dataloader = DataLoader(test_data, batch_size=8)

In [18]:
train_data[0]

(tensor([    0, 19647,  2079,  4674,  9034,  2024,  2138,  3756,  6233,  4693,
          2069,  3899,  2205,  2259, 10460, 19861,    14, 23740,  2283,  2369,
            30,  9775,  2252,  2532,  2081,  2110,    12, 15760,  2238,  2341,
            30, 22860,  2254,    13, 23740,  2418,  2170,  4340,  4162, 12765,
          2079, 23740, 19861,  2052,  2307,  1485,  2382,  2119, 23740,  2259,
          3894, 19521,  1513,  2118,  1380,  2053,    14,  1163,  5189,  2259,
            23,  2210,  2165,  4208,  2138,  3894, 11800,    18,     2,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,  

In [19]:
# model = RobertaForSequenceClassification.from_pretrained(HUGGINGFACE_MODEL_PATH, return_dict=False)
model = RobertaForSequenceClassification.from_pretrained(HUGGINGFACE_MODEL_PATH, num_labels=14, problem_type="multi_label_classification", return_dict=False)

Some weights of the model checkpoint at klue/roberta-small were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.decoder.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-small and are newly initialized: ['classifier.dense.bias', 'classifier.out_proj.weight', 'class

In [20]:
from transformers import AdamW, get_linear_schedule_with_warmup

optimizer = AdamW(model.parameters(),
                  lr = 2e-5, 
                  eps = 1e-8 
                )

# Number of training epochs
epochs = 4

# Total number of training steps is number of batches * number of epochs.
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler
scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)
loss_fn = nn.CrossEntropyLoss(reduction='sum')
model.cuda()

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerN

In [21]:
batch_size = 8

In [22]:
def calc_accuracy(X, Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy() / max_indices.size()[0]
    return train_acc


In [46]:
# Training
import time
# Store the average loss after each epoch 
loss_values = []
# number of total steps for each epoch
print('total steps per epoch: ',  len(train_dataloader) / batch_size)
# looping over epochs
for epoch_i in range(0, epochs):
    
    torch.cuda.empty_cache()
    gc.collect()
    
    print('training on epoch: ', epoch_i)
    # set start time 
    t0 = time.time()
    # reset total loss
    total_loss = 0
    train_acc = 0.0
    # model in training 
    model.train()
    # loop through batch 
    for step, batch in enumerate(train_dataloader):
        # Progress update every 50 step 
        if step % 50 == 0 and not step == 0:
            print('training on step: ', step)
#             print('total time used is: {0:.2f} s'.format(time.time() - t0))
        # load data from dataloader 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].float().to(device)
        # clear any previously calculated gradients 
        model.zero_grad()
        # get outputs
        outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
        # get loss
        loss = outputs[0]
        # total loss
        total_loss += loss.item()
        # clip the norm of the gradients to 1.0.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        # update optimizer
        optimizer.step()
        # update learning rate 
        scheduler.step()
        
    # Calculate the average loss over the training data.
    avg_train_loss = total_loss / len(train_dataloader)
    # Store the loss value for plotting the learning curve.
    loss_values.append(avg_train_loss)
    print("average training loss: {0:.2f}".format(avg_train_loss))
    
    # Test
    import numpy as np
    t0 = time.time()
    # model in validation mode
    model.eval()
    # save prediction
    predictions,true_labels =[],[]
    # evaluate data for one epoch
    for batch in test_dataloader:
        # Add batch to GPU
        batch = tuple(t.to(device) for t in batch)
        # Unpack the inputs from our dataloader
        b_input_ids, b_input_mask, b_labels = batch
        # validation
        with torch.no_grad():
            outputs = model(b_input_ids,
                            token_type_ids=None,
                            attention_mask=b_input_mask)
        # get output
        logits = outputs[0]
        # move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        final_prediction = np.argmax(logits, axis=-1).flatten()
        label_num = np.argmax(label_ids, axis=-1).flatten()
        predictions.append(final_prediction)
        true_labels.append(label_num)

    print('total time used is: {0:.2f} s'.format(time.time() - t0))

total steps per epoch:  1612.875
training on epoch:  0
training on step:  50
total time used is: 2.67 s
training on step:  100
total time used is: 5.29 s
training on step:  150
total time used is: 7.92 s
training on step:  200
total time used is: 10.56 s
training on step:  250
total time used is: 13.22 s
training on step:  300
total time used is: 15.87 s
training on step:  350
total time used is: 18.53 s
training on step:  400
total time used is: 21.19 s
training on step:  450
total time used is: 23.87 s
training on step:  500
total time used is: 26.56 s
training on step:  550
total time used is: 29.25 s
training on step:  600
total time used is: 31.93 s
training on step:  650
total time used is: 34.63 s
training on step:  700
total time used is: 37.34 s
training on step:  750
total time used is: 40.05 s
training on step:  800
total time used is: 42.76 s
training on step:  850
total time used is: 45.46 s
training on step:  900
total time used is: 48.18 s
training on step:  950
total ti

KeyboardInterrupt: 

In [43]:
# convert numeric label to string
final_prediction_list = label_encoder.inverse_transform(np.concatenate(predictions))
final_truelabel_list = label_encoder.inverse_transform(np.concatenate(true_labels))

final_prediction_list

array(['기타', '기타', '기타', ..., '기타', '기타', '기타'], dtype=object)

In [44]:
from sklearn.metrics import confusion_matrix, classification_report
cr = classification_report(final_truelabel_list, 
                           final_prediction_list, 
                           output_dict=False)
print(cr)

              precision    recall  f1-score   support

        경제금융       0.00      0.00      0.00      1493
        교육과학       0.00      0.00      0.00       809
        교통물류       0.00      0.00      0.00      1054
          기타       0.00      0.80      0.00         5
        문화관광       0.00      0.00      0.00      1291
        보건의료       0.00      0.00      0.00       457
        사회복지       0.08      0.27      0.12       390
        식품건강       0.00      0.00      0.00       798
        재난안전       0.01      0.00      0.01       618
        제조소비       0.00      0.00      0.00      1119
        주택토지       0.00      0.00      0.00       283
      통일외교안보       0.00      0.00      0.00        33
        행정법률       0.00      0.00      0.00      1965
        환경기상       0.00      0.00      0.00      1186

    accuracy                           0.01     11501
   macro avg       0.01      0.08      0.01     11501
weighted avg       0.00      0.01      0.00     11501



/home/keti/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/keti/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/keti/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [40]:
torch.save(model, "roberta_small.pt")
torch.save(model.state_dict(),"roberta_small_state_dict.pt")
torch.save({"model": model.state_dict(), "optimizer": optimizer.state_dict()},"roberta_small_all.tar",)